In [ ]:
import sys
sys.path.append('../')
import numpy as np
import open3d as o3d
import torch
import seaborn as sns
import json

from scene.gaussian_model import GaussianModel
from scene.artgs import ArtGS

In [21]:
def visualize_point_cloud(xyz, rgb=None, save_path=None):
    """
    xyz: (N, 3) numpy array
    rgb: (N, 3) numpy array
    """
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(xyz)
    if rgb is not None:
        point_cloud.colors = o3d.utility.Vector3dVector(rgb)
    o3d.visualization.draw_geometries([point_cloud], )
    if save_path is not None:
        o3d.io.write_point_cloud(save_path, point_cloud)

In [22]:
dataset = "artgs"
subset = "sapien"
scene = "table_31249"
dir = f"../outputs/{dataset}/{subset}/{scene}"
name = "artgs"
local_dir = f'{dir}/{name}'
print(local_dir)

../outputs/artgs/sapien/table_31249/artgs


In [23]:
joint_types = json.load(open(f'../arguments/joint_types_cgs.json', 'r'))[dataset][subset][scene]
print(joint_types)
class Args(object):
    def __init__(self):
        self.slot_size = 32
        self.joint_types = joint_types
        self.num_slots = len(self.joint_types.split(','))
        self.gumbel = True
        self.scale_factor = 1.
        self.use_art_type_prior = True
        self.shift_weight = 0.5
        self.tau_decay_steps = 10000
args = Args()
joint_types = args.joint_types.split(',')
pallete = np.array(sns.color_palette("hls", args.num_slots))
pallete[0] = [0, 0, 0]

s,r,r,p,p


## Visualize Part Segmentation and Joint Axis

In [ ]:
loaded_iter = 'best'
ckpt_dir = f'iteration_{loaded_iter}'
gs = GaussianModel(3)
gs.load_ply(f'{local_dir}/point_cloud/{ckpt_dir}/point_cloud.ply')
deform = ArtGS(args).cuda()
deform.load_state_dict(torch.load(f'{local_dir}/deform/{ckpt_dir}/deform.pth', weights_only=True))
deform.update(20000)
deform.eval()   
with torch.no_grad():
    xyz = gs.get_xyz.cpu().numpy()
    print(xyz.shape)
    mask = deform.get_mask(gs.get_xyz).argmax(1).cpu().numpy()
    color = pallete[mask]
    center = deform.seg_model.center.detach().cpu().numpy()
    xyz_center = (center[None] + np.random.randn(1000, center.shape[0], 3) * 0.01).reshape(-1, 3)
    rgb_center = pallete[None].repeat(1000, 0).reshape(-1, 3)

    joint_info_list = deform.get_joint_param(joint_types[1:])
    centers = deform.seg_model.center.detach().cpu().numpy()
    axises = []
    for i, joint_info in enumerate(joint_info_list):
        direction = joint_info['axis_direction']
        pos = joint_info['axis_position']
        if joint_types[i+1] == 'p':
            pos = centers[i+1]
        pos += direction * np.dot(direction, centers[i+1] - pos)
        # visualize the joint
        axis_points = pos[None] + direction[None] * np.linspace(-1, 1, 1000)[:, None]
        axis_colors = pallete[i+1][None].repeat(1000, 0)
        axis = np.concatenate([axis_points, axis_colors], axis=1)
        axises.append(axis)
    axis = np.concatenate(axises)
    
    xyz_vis = np.concatenate([xyz, xyz_center])
    color_vis = np.concatenate([color, rgb_center])
    visualize_point_cloud(xyz_vis, color_vis)

NameError: name 'GaussianModel' is not defined